In [133]:
import numpy as np
import tensorflow as tf


In [134]:
import gzip
import pandas as pd
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

train_df = getDF('train.json.gz')
test_df = getDF('test_Helpful.json.gz')

In [135]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower = True, split = ' ')

In [136]:
tokenizer.fit_on_texts(train_df.reviewText.tolist())

In [137]:
train_text = tokenizer.texts_to_sequences(train_df.reviewText.tolist())
test_text = tokenizer.texts_to_sequences(test_df.reviewText.tolist())

In [138]:
from keras.preprocessing.sequence import pad_sequences

padded_train = pad_sequences(maxlen = 128, sequences = train_text)
padded_text  = pad_sequences(maxlen = 128,sequences = test_text)

# With batch size 1 we do not need padding

In [163]:
import keras.models
from keras import layers

model = keras.Sequential()

# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 256.

model.add(layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=256))

# Add a LSTM layer with 256 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.from keras import backend as K
K.tensorflow_backend._get_available_gpus()
model.add(layers.Dense(32,activation = 'relu'))
model.add(layers.Dense(1))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 256)         17605120  
_________________________________________________________________
lstm_16 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dense_22 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 33        
Total params: 17,806,401
Trainable params: 17,806,401
Non-trainable params: 0
_________________________________________________________________


In [164]:
helpfulNess_train = pd.DataFrame(train_df.helpful.apply(lambda x: [x['outOf'],x['nHelpful']]).to_list(),columns = ['outOf','nHelpful'])

helpfulNess_test = pd.DataFrame(test_df.helpful.apply(lambda x: [x['outOf']]).to_list(),columns = ['outOf'])

In [165]:
model.compile('rmsprop','mse',metrics = ['mae'])

In [166]:
model.fit(padded_train,helpfulNess_train['nHelpful'].to_numpy(),epochs=5,batch_size=64)

/home/akash/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
200000/200000 [==============================] - 738s 4ms/step - loss: 33.4351 - mae: 1.5779
Epoch 2/5
200000/200000 [==============================] - 759s 4ms/step - loss: 33.0538 - mae: 1.5514
Epoch 3/5
200000/200000 [==============================] - 740s 4ms/step - loss: 32.9199 - mae: 1.5406
Epoch 4/5
200000/200000 [==============================] - 2104s 11ms/step - loss: 32.8844 - mae: 1.5407
Epoch 5/5
200000/200000 [==============================] - 865s 4ms/step - loss: 32.6449 - mae: 1.5277


In [168]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17997439007480660996
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10811109098388442549
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15197652865059958853
physical_device_desc: "device: XLA_GPU device"
]
